In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score
import numpy as np

# Veri setini yükleme
train_data_dir = r"C:\Users\golde\Desktop\data2\Data\train"
validation_data_dir = r"C:\Users\golde\Desktop\data2\Data\valid"
test_data_dir = r"C:\Users\golde\Desktop\data2\Data\test"

# Veri ön işleme işlemleri için ImageDataGenerator kullanımı
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)


model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False


model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])


history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))


test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


Y_pred = model.predict(test_generator)
y_pred = np.round(Y_pred).flatten()
y_true = test_generator.classes

# Performans metriklerinin hesaplanması
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)
auc = roc_auc_score(y_true, Y_pred)
f1 = f1_score(y_true, y_pred)

print('Confusion Matrix:\n', cm)
print('Classification Report:\n', cr)
print('AUC Score:', auc)
print('F1 Score:', f1)


model.save("resnet_model.h5")
print("Model kaydedildi.")

